In [1]:
! pip install -q kaggle

In [2]:
!pip install transformers


     |████████████████████████████████| 2.6 MB 7.7 MB/s 
     |████████████████████████████████| 636 kB 58.7 MB/s 
     |████████████████████████████████| 3.3 MB 60.9 MB/s 
     |████████████████████████████████| 895 kB 64.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install unidecode
!pip install torch
!pip install tqdm

     |████████████████████████████████| 241 kB 8.4 MB/s 


In [46]:
from google.colab import files
from tqdm import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gabib3b","key":"817d7e169db4cbef867b22907320144c"}'}

In [6]:
! mkdir ~/.kaggle

In [7]:
! cp kaggle.json ~/.kaggle/

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
! kaggle competitions download -c home-depot-product-search-relevance

 33% 9.00M/27.2M [00:00<00:00, 29.3MB/s]
100% 27.2M/27.2M [00:00<00:00, 61.4MB/s]
100% 4.74M/4.74M [00:00<00:00, 48.5MB/s]

  0% 0.00/227k [00:00<?, ?B/s]
100% 227k/227k [00:00<00:00, 70.0MB/s]
  0% 0.00/105k [00:00<?, ?B/s]
100% 105k/105k [00:00<00:00, 112MB/s]
 95% 33.0M/34.8M [00:01<00:00, 26.4MB/s]
100% 34.8M/34.8M [00:01<00:00, 30.0MB/s]
  0% 0.00/2.51M [00:00<?, ?B/s]
100% 2.51M/2.51M [00:00<00:00, 151MB/s]


In [29]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import nltk 
import unidecode
import re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
!ls

attributes.csv.zip	      relevance_instructions.docx  test.csv.zip
kaggle.json		      sample_data		   train.csv.zip
product_descriptions.csv.zip  sample_submission.csv.zip


In [12]:
att_df= pd.read_csv('attributes.csv.zip')
desc_df= pd.read_csv('product_descriptions.csv.zip')
train_df= pd.read_csv('train.csv.zip', encoding='latin-1')
test_df = pd.read_csv('test.csv.zip',encoding='latin-1')


In [14]:
desc_df = desc_df.fillna(0)
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
att_df = att_df.fillna(0)

desc_df['product_uid'] = desc_df['product_uid'].astype(np.int64)
train_df['product_uid'] = train_df['product_uid'].astype(np.int64)
test_df['product_uid'] = test_df['product_uid'].astype(np.int64)
att_df['product_uid'] = att_df['product_uid'].astype(np.int64)

In [16]:
train_df.sample(2)

,id,product_uid,product_title,search_term,relevance
26071,79939,123316,KOHLER CLC 30 in. x 26 in. Recessed or Surface...,bath si k cabinets,2.67
36027,110117,135898,Burpee Spinach Bloomsdale Long-Standing Seed,spinach,3.00


In [ ]:
np.min(train_df['relevance'].tolist()), np.max(train_df['relevance'].tolist())

(1.0, 3.0)

In [17]:
train_df['my_relevance'] = train_df['relevance'].apply(lambda x: (x - 1.0)/2.0)

In [18]:
np.min(train_df['my_relevance'].tolist()), np.max(train_df['my_relevance'].tolist())

(0.0, 1.0)

In [19]:
train_df['my_relevance2'] = train_df['my_relevance'].apply(lambda x: ((x - 0.0)/1.0) * 2.0 + 1)

In [20]:
train_df[['relevance', 'my_relevance', 'my_relevance2']].sample(4)

,relevance,my_relevance,my_relevance2
52071,2.33,0.665,2.33
12742,1.67,0.335,1.67
13883,3.00,1.000,3.00
36757,1.00,0.000,1.00


In [ ]:
train_df.describe()

,id,product_uid,relevance
count,74067.000000,74067.000000,74067.000000
mean,112385.709223,142331.911553,2.381634
std,64016.573650,30770.774864,0.533984
min,2.000000,100001.000000,1.000000
25%,57163.500000,115128.500000,2.000000
50%,113228.000000,137334.000000,2.330000
75%,168275.500000,166883.500000,3.000000
max,221473.000000,206650.000000,3.000000


In [23]:
train_df.drop_duplicates(["product_uid"]).shape[0]/len(train_df)

0.7380749861611784

In [ ]:
# pd.merge(train_df, desc_df, how='left', on='product_uid').shape[0]

74067

In [24]:
train_df.shape[0]

74067

In [25]:
EMPTY_STRING = ''
SPACE = ' '
def clean_text(string):
  if string is None:
    return None

  string = string.strip().lower()
  string = unidecode.unidecode(string)
  string = string.replace("'", EMPTY_STRING)
  string = re.sub(r"\'n", SPACE, string)
  string = re.sub(r"\'s", " 's", string)
  string = re.sub(r"\'ve", " 've", string)
  string = re.sub(r"n\'t", " n't", string)
  string = re.sub(r"\'re", " 're", string)
  string = re.sub(r"\'d", " 'd", string)
  string = re.sub(r"\'ll", " 'll", string)
  string = re.sub(r",", " , ", string)
  string = re.sub(r"!", " ! ", string)
  string = re.sub(r"\(", " ( ", string)
  string = re.sub(r"\)", " ) ", string)
  string = re.sub(r"\?", r" \? ", string)
  string = re.sub(r"\s{2,}", SPACE, string)
  string = re.sub(r" +", SPACE, string)
  string = re.sub(r"/", " / ", string)
  string = re.sub(r"\|", " | ", string)
  string = re.sub(r"-", " - ", string)
  string = re.sub(r"\+", " + ", string)
  string = re.sub(r"\\\\", SPACE, string)

  return string.strip()


In [36]:
_characters_to_clean = string.punctuation + "''"
EN_STOP_WORDS = set(stopwords.words('english')) - set(['can', 'no', 'up', 'against', 'after', 'before', 'below'])

PUNCTUATIONS = '!"#$&\'()*+,-./:;<=>?@[\\]^_`{|}~'

PUNCTUATIONS_PATTERN = re.compile("[{}]+$".format(re.escape(PUNCTUATIONS)))

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def _is_punctuation_only(word: str):
    return PUNCTUATIONS_PATTERN.match(word)

def filter_ale_punctuation(words):
    return [word for word in words if not _is_punctuation_only(word)] if words is not None else None

def tokenize(text):
  
  processed_text = clean_text(text)
  tokens = nltk.word_tokenize(processed_text)

  
  tokens = [stemmer.stem(word) for word in tokens]
  tokens = [word for word in tokens if word not in _characters_to_clean and word not in EN_STOP_WORDS]
  tokens = filter_ale_punctuation(tokens)
  tokens = [re.sub("\d", "#", word) for word in tokens]

  return tokens if len(tokens) > 0 else None


In [40]:
# tokenize("he went home 123 times, before we --gabi op. -;[]")

['went', 'home', '###', 'time', 'befor', 'gabi', 'op']

,id,product_uid,product_title,search_term,relevance,my_relevance,my_relevance2
60155,181609,175644,Pergo Presto Walden Oak Laminate Flooring - 5 ...,high density sound board,1.67,0.335,1.67
17811,54945,114385,Hot Shot 1 gal. Ready-to-Use Bed Bug Killer,diatemaceous earth,1.67,0.335,1.67
19121,59070,115749,Eglo Tosca 3-Light Antique Brown Hanging Islan...,dining room lighting,2.67,0.835,2.67
66166,199007,188214,Pine-Sol 60 oz. Lemon Fresh All-Purpose Cleaner,show me all 60 inch vaniteis,1.67,0.335,1.67
7207,22566,104340,ClosetMaid Preloaded Wall Brackets for SuperSl...,closet maid wire shelving,2.33,0.665,2.33
18348,56633,114928,48 in. x 96 in. x .157 in. White Corrugated Pl...,sheet plastic,3.00,1.000,3.00
70787,212011,198637,Zurn EZ Flush Valve with Chrome Plated Plastic...,zurn ez-flush,3.00,1.000,3.00
19002,58701,115650,ODL White Cordless Add On Enclosed Aluminum Bl...,72' wide mini blinds,2.33,0.665,2.33
55364,167753,166555,"Panasonic 12,000 BTU 1 Ton Ductless Mini Split...",out side heat pump covers,1.67,0.335,1.67
17863,55116,114435,Southwire 8-4 SOOW Multi-Use Electrical Cord -...,8 foot flour sent,1.00,0.000,1.00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


0.9997299742125373

In [48]:
train_df['product_title_tokens'] = train_df['product_title'].progress_apply(tokenize)

100%|██████████| 74067/74067 [00:35<00:00, 2096.44it/s]


In [47]:
train_df['search_term_tokens'] = train_df['search_term'].progress_apply(tokenize)

100%|██████████| 74067/74067 [00:14<00:00, 5141.27it/s]


In [ ]:
pd.set_option("max_colwidth", -1)

original_train_df_length = len(train_df)
train_df = train_df[train_df['search_term_tokens'].notnull()]
len(train_df)/original_train_df_length

In [50]:
train_df.sample(3)

,id,product_uid,product_title,search_term,relevance,my_relevance,my_relevance2,product_title_tokens,search_term_tokens
58288,176171,171947,GE 6.6 cu. ft. Double Oven Electric Range with...,stainless steel electric range and microwave,2.67,0.835,2.67,"[ge, #.#, cu, ft., doubl, oven, electr, rang, ...","[stainless, steel, electr, rang, microwav]"
51449,156266,159633,Elkay Lustertone Top Mount Stainless Steel 15 ...,elkay bar sink,3.00,1.000,3.00,"[elkay, lusterton, top, mount, stainless, stee...","[elkay, bar, sink]"
14509,44793,111005,LEXAN Thermoclear 48 in. x 96 in. x 1/4 in. Cl...,plasticl panels,2.00,0.500,2.00,"[lexan, thermoclear, ##, x, ##, x, #, #, clear...","[plasticl, panel]"


In [66]:
all_words = [word for words in train_df['search_term_tokens'].tolist() for word in words]
all_words.extend([word for words in train_df['product_title_tokens'].tolist() for word in words])

vocab = set(all_words)
len(vocac), len(all_words)

(16345, 1055648)

In [71]:
WORD_REPLACE_MENT = '<word>'

In [67]:
# from collections import Counter
# sentenses = train_df['search_term_tokens'].tolist()
# sentenses.extend(train_df['product_title_tokens'].tolist())
counts = Counter(all_words)


In [72]:
MIN_WORD_COUNT = 3

In [73]:
train_df['product_title_tokens'] = train_df['product_title_tokens'].progress_apply(lambda words: [word if counts[word] >= MIN_WORD_COUNT else WORD_REPLACE_MENT for word in words])


100%|██████████| 74047/74047 [00:00<00:00, 116211.52it/s]


In [76]:
train_df['search_term_tokens'] = train_df['search_term_tokens'].progress_apply(lambda words: [word if counts[word] >= MIN_WORD_COUNT else WORD_REPLACE_MENT for word in words])


100%|██████████| 74047/74047 [00:00<00:00, 428675.82it/s]


In [77]:
train_df[train_df['product_title_tokens'].map(len) == 0]

,id,product_uid,product_title,search_term,relevance,my_relevance,my_relevance2,product_title_tokens,search_term_tokens


In [78]:
train_df[train_df['search_term_tokens'].map(len) == 0]

,id,product_uid,product_title,search_term,relevance,my_relevance,my_relevance2,product_title_tokens,search_term_tokens
